In [1]:
'''
This file plots CMIP5 RCP - HIST over South Australia
/Users/earl/Desktop/Yang
and places the outputs in
/Users/earl/Dropbox/CMIP5/figures

Earl Duran 
created: 19-Mar-18
e.duran@unsw.edu.au
'''

import cosima_cookbook as cc
import os
import xarray as xr
import numpy as np
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import matplotlib
import sys
import pickle
import itertools
def find_nearest_index(array, value):
    return int((np.abs(array - value)).argmin())
from scipy import interpolate
from scipy import stats
from dask.distributed import Client

import cartopy.crs as ccrs
import cartopy.feature as cft
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
def arange(start,end,stride=1):
    return np.arange(start,end+0.00000001,stride)

import sys, os
sys.path.append(os.path.join(os.getcwd(), '..'))  # so we can import ../exptdata
import exptdata
print('Available exptdata keys: ', [k for k in exptdata.exptdict.keys()])

session = cc.database.create_session()

Available exptdata keys:  ['1deg', '025deg', '01deg']


In [2]:
%%javascript
IPython.notebook.kernel.execute('nb_name = ' + '"' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [3]:
client = Client()
client

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/distributed/dashboard/core.py:72: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


Client Scheduler: tcp://127.0.0.1:43660 Dashboard: http://127.0.0.1:41310/status,Cluster Workers: 4 Cores: 8 Memory: 33.67 GB


In [4]:
# input_path = '/g/data/e14/erd561/NOAA_OI_SST_V2/'

output_path = '/g/data/e14/erd561/Australia_2004-2017/'

In [5]:
%%time
latS = -70
latN = 20
lonW = 90-360
lonE = 180-360
ekey='1deg'
expt = exptdata.exptdict[ekey]['expt']
print(expt)

temp_z0_1_mod_month = cc.querying.getvar(expt=expt,
                                     session=session,
                                     ncfile='ocean.nc',
                                     variable='temp').\
sel(yt_ocean=slice(latS,latN)).\
sel(xt_ocean=slice(lonW,lonE)).\
sel(st_ocean=0, method='nearest')-273.15
print(temp_z0_1_mod_month)

temp_z0_1_mod_month.load()
print(temp_z0_1_mod_month)

xt_ocean = temp_z0_1_mod_month.xt_ocean
xt_ocean_corrected = xt_ocean + 360
temp_z0_1_mod_month = temp_z0_1_mod_month.assign_coords(xt_ocean=xt_ocean_corrected)
print(temp_z0_1_mod_month)

temp_z0_1_mod = temp_z0_1_mod_month.groupby('time.year').mean('time')
print(temp_z0_1_mod)

temp_z0_1_mod = temp_z0_1_mod.sel(year=slice(2004+60*4,2017+60*4))
print(temp_z0_1_mod)

1deg_jra55v13_iaf_spinup1_B1


/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/cosima_cookbook/querying.py:96: FutureWarning: In xarray version 0.14 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  preprocess=lambda d: d[variable].to_dataset() if variable not in d.coords else d)
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/backends/api.py:934: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option t

<xarray.DataArray 'temp' (time: 300, yt_ocean: 160, xt_ocean: 90)>
dask.array<sub, shape=(300, 160, 90), dtype=float32, chunksize=(1, 124, 90), chunktype=numpy.ndarray>
Coordinates:
  * xt_ocean  (xt_ocean) float64 -269.5 -268.5 -267.5 ... -182.5 -181.5 -180.5
  * yt_ocean  (yt_ocean) float64 -69.63 -69.18 -68.71 ... 17.85 18.59 19.36
    st_ocean  float64 1.152
  * time      (time) object 1958-07-02 12:00:00 ... 2257-07-02 12:00:00
<xarray.DataArray 'temp' (time: 300, yt_ocean: 160, xt_ocean: 90)>
array([[[       nan,        nan,        nan, ..., -1.3452759,
         -1.3441772, -1.342804 ],
        [       nan,        nan,        nan, ..., -1.3128967,
         -1.3044739, -1.3048401],
        [       nan,        nan,        nan, ..., -1.2649231,
         -1.258606 , -1.2553406],
        ...,
        [27.779205 , 27.703247 , 27.695343 , ..., 26.297943 ,
         26.258667 , 26.213867 ],
        [27.70932  , 27.692932 , 27.660675 , ..., 26.196869 ,
         26.152313 , 26.096619 ],
   

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


In [6]:
year = temp_z0_1_mod.year
lat_1_mod = temp_z0_1_mod.yt_ocean
lon_1_mod = temp_z0_1_mod.xt_ocean

temp_z0_1_trans_mod = temp_z0_1_mod
print(np.shape(temp_z0_1_trans_mod))
temp_z0_1_slope_mod = xr.DataArray(
    np.zeros([np.shape(lat_1_mod)[0], np.shape(lon_1_mod)[0]]), dims=(
    'lat_1', 'lon_1'), coords=[lat_1_mod, lon_1_mod], name='temp_z0_1')
temp_z0_1_p_value_mod = xr.DataArray(
    np.zeros([np.shape(lat_1_mod)[0], np.shape(lon_1_mod)[0]]), dims=(
    'lat_1', 'lon_1'), coords=[lat_1_mod, lon_1_mod], name='temp_z0_1')
temp_z0_1_std_err_mod = xr.DataArray(
    np.zeros([np.shape(lat_1_mod)[0], np.shape(lon_1_mod)[0]]), dims=(
    'lat_1', 'lon_1'), coords=[lat_1_mod, lon_1_mod], name='temp_z0_1')
for iid, i in enumerate(lat_1_mod):
    for jid, j in enumerate(lon_1_mod):
        temp_z0_1_slope_mod[iid, jid], _, _, temp_z0_1_p_value_mod[iid, jid], temp_z0_1_std_err_mod[iid, jid] = \
        stats.linregress(year, temp_z0_1_trans_mod[:, iid, jid])
    print('lat_1_mod ' + str(np.array(i)))
print(temp_z0_1_slope_mod)

(14, 160, 90)
lat_1_mod -69.6292858330716
lat_1_mod -69.17632154294074
lat_1_mod -68.70935410409521
lat_1_mod -68.22843658249317
lat_1_mod -67.73301847540898
lat_1_mod -67.2232002021303
lat_1_mod -66.69847936238045
lat_1_mod -66.15900514695375
lat_1_mod -65.60432451496254
lat_1_mod -65.03463652231858
lat_1_mod -64.44953841598465
lat_1_mod -63.84927987875662
lat_1_mod -63.233509039251814
lat_1_mod -62.602526632003375
lat_1_mod -61.95603192247815
lat_1_mod -61.29437678205883
lat_1_mod -60.61731152794955
lat_1_mod -59.92523891318751
lat_1_mod -59.21795988186098
lat_1_mod -58.49592747485757
lat_1_mod -57.75899250138302
lat_1_mod -57.00765736171395
lat_1_mod -56.24182163656275
lat_1_mod -55.46203582865502
lat_1_mod -54.66824687203264
lat_1_mod -53.86105179481633
lat_1_mod -53.04044315107174
lat_1_mod -52.20706260764199
lat_1_mod -51.36094630171701
lat_1_mod -50.50277835512628
lat_1_mod -49.63263616124607
lat_1_mod -48.751243830625256
lat_1_mod -47.858717411335334
lat_1_mod -46.9558182702592

In [7]:
%%time
latS = -70
latN = 20
lonW = 90-360
lonE = 180-360
ekey='025deg'
expt = exptdata.exptdict[ekey]['expt']
print(expt)

temp_z0_025_mod_month = cc.querying.getvar(expt=expt,
                                     session=session,
                                     ncfile='ocean.nc',
                                     variable='temp').\
sel(yt_ocean=slice(latS,latN)).\
sel(xt_ocean=slice(lonW,lonE)).\
sel(st_ocean=0, method='nearest')-273.15
print(temp_z0_025_mod_month)

temp_z0_025_mod_month.load()
print(temp_z0_025_mod_month)

xt_ocean = temp_z0_025_mod_month.xt_ocean
xt_ocean_corrected = xt_ocean + 360
temp_z0_025_mod_month = temp_z0_025_mod_month.assign_coords(xt_ocean=xt_ocean_corrected)
print(temp_z0_025_mod_month)

temp_z0_025_mod = temp_z0_025_mod_month.groupby('time.year').mean('time')
print(temp_z0_025_mod)

temp_z0_025_mod = temp_z0_025_mod.sel(year=slice(2004+60*4,2017+60*4))
print(temp_z0_025_mod)

025deg_jra55v13_iaf_gmredi6


/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/cosima_cookbook/querying.py:96: FutureWarning: In xarray version 0.14 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  preprocess=lambda d: d[variable].to_dataset() if variable not in d.coords else d)
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/backends/api.py:934: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option t

<xarray.DataArray 'temp' (time: 300, yt_ocean: 475, xt_ocean: 360)>
dask.array<sub, shape=(300, 475, 360), dtype=float32, chunksize=(1, 216, 248), chunktype=numpy.ndarray>
Coordinates:
  * xt_ocean  (xt_ocean) float64 -269.9 -269.6 -269.4 ... -180.6 -180.4 -180.1
  * yt_ocean  (yt_ocean) float64 -69.99 -69.88 -69.78 ... 19.49 19.72 19.96
    st_ocean  float64 1.152
  * time      (time) object 1958-06-30 12:00:00 ... 2257-06-30 12:00:00
<xarray.DataArray 'temp' (time: 300, yt_ocean: 475, xt_ocean: 360)>
array([[[       nan,        nan,        nan, ..., -1.3691101,
         -1.3626709, -1.355896 ],
        [       nan,        nan,        nan, ..., -1.3651428,
         -1.3577881, -1.3512878],
        [       nan,        nan,        nan, ..., -1.3556519,
         -1.3484192, -1.3435669],
        ...,
        [27.802582 , 27.755707 , 27.710144 , ..., 25.934357 ,
         25.9169   , 25.897644 ],
        [27.709442 , 27.649994 , 27.596466 , ..., 25.882812 ,
         25.862396 , 25.845978 ],

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


<xarray.DataArray 'temp' (year: 300, yt_ocean: 475, xt_ocean: 360)>
array([[[       nan,        nan,        nan, ..., -1.3691101,
         -1.3626709, -1.355896 ],
        [       nan,        nan,        nan, ..., -1.3651428,
         -1.3577881, -1.3512878],
        [       nan,        nan,        nan, ..., -1.3556519,
         -1.3484192, -1.3435669],
        ...,
        [27.802582 , 27.755707 , 27.710144 , ..., 25.934357 ,
         25.9169   , 25.897644 ],
        [27.709442 , 27.649994 , 27.596466 , ..., 25.882812 ,
         25.862396 , 25.845978 ],
        [27.600006 , 27.545258 , 27.514069 , ..., 25.834686 ,
         25.809082 , 25.799255 ]],

       [[       nan,        nan,        nan, ..., -1.6878967,
         -1.6752625, -1.658081 ],
        [       nan,        nan,        nan, ..., -1.6947937,
         -1.6841736, -1.6699829],
        [       nan,        nan,        nan, ..., -1.6965942,
         -1.6852722, -1.6719666],
        ...,
        [27.432068 , 27.406036 , 27.3960

In [8]:
lat_025_mod = temp_z0_025_mod.yt_ocean
lon_025_mod = temp_z0_025_mod.xt_ocean

temp_z0_025_trans_mod = temp_z0_025_mod
print(np.shape(temp_z0_025_trans_mod))
temp_z0_025_slope_mod = xr.DataArray(
    np.zeros([np.shape(lat_025_mod)[0], np.shape(lon_025_mod)[0]]), dims=(
    'lat_025', 'lon_025'), coords=[lat_025_mod, lon_025_mod], name='temp_z0_025')
temp_z0_025_p_value_mod = xr.DataArray(
    np.zeros([np.shape(lat_025_mod)[0], np.shape(lon_025_mod)[0]]), dims=(
    'lat_025', 'lon_025'), coords=[lat_025_mod, lon_025_mod], name='temp_z0_025')
temp_z0_025_std_err_mod = xr.DataArray(
    np.zeros([np.shape(lat_025_mod)[0], np.shape(lon_025_mod)[0]]), dims=(
    'lat_025', 'lon_025'), coords=[lat_025_mod, lon_025_mod], name='temp_z0_025')
for iid, i in enumerate(lat_025_mod):
    for jid, j in enumerate(lon_025_mod):
        temp_z0_025_slope_mod[iid, jid], _, _, temp_z0_025_p_value_mod[iid, jid], temp_z0_025_std_err_mod[iid, jid] = \
        stats.linregress(year, temp_z0_025_trans_mod[:, iid, jid])
    print('lat_025_mod ' + str(np.array(i)))
print(temp_z0_025_slope_mod)

(14, 475, 360)
lat_025_mod -69.98914896732052
lat_025_mod -69.8835503781156
lat_025_mod -69.77795178891068
lat_025_mod -69.67235319970577
lat_025_mod -69.56675461050085
lat_025_mod -69.46115602129593
lat_025_mod -69.35555743209102
lat_025_mod -69.2499588428861
lat_025_mod -69.14436025368119
lat_025_mod -69.03876166447627
lat_025_mod -68.93316307527135
lat_025_mod -68.82756448606644
lat_025_mod -68.72196589686152
lat_025_mod -68.6163673076566
lat_025_mod -68.51076871845169
lat_025_mod -68.40517012924677
lat_025_mod -68.29957154004185
lat_025_mod -68.19397295083694
lat_025_mod -68.08837436163202
lat_025_mod -67.9827757724271
lat_025_mod -67.87717718322219
lat_025_mod -67.77157859401727
lat_025_mod -67.66598000481235
lat_025_mod -67.56038141560744
lat_025_mod -67.45478282640252
lat_025_mod -67.3491842371976
lat_025_mod -67.24358564799269
lat_025_mod -67.13798705878777
lat_025_mod -67.03238846958286
lat_025_mod -66.92678988037794
lat_025_mod -66.82119129117302
lat_025_mod -66.7155927019681

lat_025_mod -30.778202070190314
lat_025_mod -30.56317292803898
lat_025_mod -30.34766755148325
lat_025_mod -30.131685174432455
lat_025_mod -29.915230577717967
lat_025_mod -29.698303070212805
lat_025_mod -29.480907483686703
lat_025_mod -29.2630432019257
lat_025_mod -29.044715107989248
lat_025_mod -28.82592266049411
lat_025_mod -28.606670794106993
lat_025_mod -28.38695904216067
lat_025_mod -28.166792391212045
lat_025_mod -27.946170449161986
lat_025_mod -27.72509825470525
lat_025_mod -27.50357549012893
lat_025_mod -27.28160724647727
lat_025_mod -27.059193280207044
lat_025_mod -26.83633873488693
lat_025_mod -26.61304344089147
lat_025_mod -26.389312594451354
lat_025_mod -26.165146099570944
lat_025_mod -25.940549205242547
lat_025_mod -25.715521888775697
lat_025_mod -25.490069451985352
lat_025_mod -25.264191945124278
lat_025_mod -25.037894722851966
lat_025_mod -24.811177907964588
lat_025_mod -24.58404690794839
lat_025_mod -24.356501917704698
lat_025_mod -24.12854839748857
lat_025_mod -23.90018

In [9]:
%%time
latS = -70
latN = 20
lonW = 90-360
lonE = 180-360
ekey='01deg'
expt = exptdata.exptdict[ekey]['expt']
print(expt)

temp_z0_01_mod_month = cc.querying.getvar(expt=expt,
                                     session=session,
                                     ncfile='ocean.nc',
                                     variable='temp').\
sel(yt_ocean=slice(latS,latN)).\
sel(xt_ocean=slice(lonW,lonE)).\
sel(st_ocean=0, method='nearest')-273.15
print(temp_z0_01_mod_month)

temp_z0_01_mod_month.load()
print(temp_z0_01_mod_month)

xt_ocean = temp_z0_01_mod_month.xt_ocean
xt_ocean_corrected = xt_ocean + 360
temp_z0_01_mod_month = temp_z0_01_mod_month.assign_coords(xt_ocean=xt_ocean_corrected)
print(temp_z0_01_mod_month)

temp_z0_01_mod = temp_z0_01_mod_month.groupby('time.year').mean('time')
print(temp_z0_01_mod)

temp_z0_01_mod = temp_z0_01_mod.sel(year=slice(2004,2017))
print(temp_z0_01_mod)

01deg_jra55v13_iaf


/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/cosima_cookbook/querying.py:96: FutureWarning: In xarray version 0.14 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  preprocess=lambda d: d[variable].to_dataset() if variable not in d.coords else d)
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/backends/api.py:934: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option t

<xarray.DataArray 'temp' (time: 396, yt_ocean: 1186, xt_ocean: 900)>
dask.array<sub, shape=(396, 1186, 900), dtype=float32, chunksize=(1, 300, 400), chunktype=numpy.ndarray>
Coordinates:
  * xt_ocean  (xt_ocean) float64 -269.9 -269.8 -269.7 ... -180.2 -180.1 -180.0
  * yt_ocean  (yt_ocean) float64 -70.0 -69.96 -69.92 ... 19.75 19.84 19.94
    st_ocean  float64 0.5413
  * time      (time) object 1985-01-14 12:00:00 ... 2017-12-14 12:00:00
<xarray.DataArray 'temp' (time: 396, yt_ocean: 1186, xt_ocean: 900)>
array([[[       nan,        nan,        nan, ..., -1.6495056,
         -1.6503601, -1.6513062],
        [       nan,        nan,        nan, ..., -1.647522 ,
         -1.6482849, -1.6491699],
        [       nan,        nan,        nan, ..., -1.6450806,
         -1.6460876, -1.6468506],
        ...,
        [25.293152 , 25.223358 , 25.163635 , ..., 25.029602 ,
         25.062805 , 25.099335 ],
        [25.10736  , 25.045837 , 24.992981 , ..., 25.03833  ,
         25.077118 , 25.118774

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


<xarray.DataArray 'temp' (year: 33, yt_ocean: 1186, xt_ocean: 900)>
array([[[       nan,        nan,        nan, ..., -1.7491251,
         -1.7482351, -1.7479095],
        [       nan,        nan,        nan, ..., -1.7536646,
         -1.7519506, -1.7504578],
        [       nan,        nan,        nan, ..., -1.757871 ,
         -1.7560654, -1.7536036],
        ...,
        [27.634636 , 27.625414 , 27.618845 , ..., 26.017271 ,
         26.008379 , 26.005022 ],
        [27.61979  , 27.614256 , 27.60399  , ..., 26.008963 ,
         26.004587 , 25.99981  ],
        [27.59926  , 27.58808  , 27.559431 , ..., 26.000448 ,
         26.000229 , 25.996185 ]],

       [[       nan,        nan,        nan, ..., -1.6110916,
         -1.6088562, -1.6058298],
        [       nan,        nan,        nan, ..., -1.6142935,
         -1.6126913, -1.610583 ],
        [       nan,        nan,        nan, ..., -1.616633 ,
         -1.6149164, -1.613645 ],
        ...,
        [27.499376 , 27.499647 , 27.4988

In [10]:
lat_01_mod = temp_z0_01_mod.yt_ocean
lon_01_mod = temp_z0_01_mod.xt_ocean

temp_z0_01_trans_mod = temp_z0_01_mod
print(np.shape(temp_z0_01_trans_mod))
temp_z0_01_slope_mod = xr.DataArray(
    np.zeros([np.shape(lat_01_mod)[0], np.shape(lon_01_mod)[0]]), dims=(
    'lat_01', 'lon_01'), coords=[lat_01_mod, lon_01_mod], name='temp_z0_01')
temp_z0_01_p_value_mod = xr.DataArray(
    np.zeros([np.shape(lat_01_mod)[0], np.shape(lon_01_mod)[0]]), dims=(
    'lat_01', 'lon_01'), coords=[lat_01_mod, lon_01_mod], name='temp_z0_01')
temp_z0_01_std_err_mod = xr.DataArray(
    np.zeros([np.shape(lat_01_mod)[0], np.shape(lon_01_mod)[0]]), dims=(
    'lat_01', 'lon_01'), coords=[lat_01_mod, lon_01_mod], name='temp_z0_01')
for iid, i in enumerate(lat_01_mod):
    for jid, j in enumerate(lon_01_mod):
        temp_z0_01_slope_mod[iid, jid], _, _, temp_z0_01_p_value_mod[iid, jid], temp_z0_01_std_err_mod[iid, jid] = \
        stats.linregress(year, temp_z0_01_trans_mod[:, iid, jid])
    print('lat_01_mod ' + str(np.array(i)))
print(temp_z0_01_slope_mod)

(14, 1186, 900)
lat_01_mod -69.99968147628742
lat_01_mod -69.95744212190694
lat_01_mod -69.91520276752645
lat_01_mod -69.87296341314597
lat_01_mod -69.83072405876548
lat_01_mod -69.788484704385
lat_01_mod -69.74624535000451
lat_01_mod -69.70400599562403
lat_01_mod -69.66176664124355
lat_01_mod -69.61952728686306
lat_01_mod -69.57728793248258
lat_01_mod -69.53504857810209
lat_01_mod -69.49280922372161
lat_01_mod -69.45056986934112
lat_01_mod -69.40833051496064
lat_01_mod -69.36609116058015
lat_01_mod -69.32385180619967
lat_01_mod -69.28161245181919
lat_01_mod -69.2393730974387
lat_01_mod -69.19713374305822
lat_01_mod -69.15489438867773
lat_01_mod -69.11265503429725
lat_01_mod -69.07041567991676
lat_01_mod -69.02817632553628
lat_01_mod -68.9859369711558
lat_01_mod -68.94369761677531
lat_01_mod -68.90145826239483
lat_01_mod -68.85921890801434
lat_01_mod -68.81697955363386
lat_01_mod -68.77474019925337
lat_01_mod -68.73250084487289
lat_01_mod -68.6902614904924
lat_01_mod -68.64802213611192

lat_01_mod -57.61555410415017
lat_01_mod -57.561954948283415
lat_01_mod -57.50827664011144
lat_01_mod -57.45451939409348
lat_01_mod -57.40068285779298
lat_01_mod -57.34676724518897
lat_01_mod -57.292772205033636
lat_01_mod -57.238697950834805
lat_01_mod -57.184544132535514
lat_01_mod -57.13031096318144
lat_01_mod -57.07599809390864
lat_01_mod -57.02160573730968
lat_01_mod -56.9671335457159
lat_01_mod -56.91258173127584
lat_01_mod -56.85794994751838
lat_01_mod -56.80323840615713
lat_01_mod -56.74844676192083
lat_01_mod -56.69357522609733
lat_01_mod -56.6386234546176
lat_01_mod -56.583591658352866
lat_01_mod -56.52847949443875
lat_01_mod -56.47328717333905
lat_01_mod -56.4180143533965
lat_01_mod -56.3626612446767
lat_01_mod -56.307227506731984
lat_01_mod -56.251713349239004
lat_01_mod -56.196118432962265
lat_01_mod -56.14044296719873
lat_01_mod -56.08468661392767
lat_01_mod -56.02884958207567
lat_01_mod -55.972931534839404
lat_01_mod -55.9169326807844
lat_01_mod -55.86085268432745
lat_01

lat_01_mod -39.95288117846647
lat_01_mod -39.87618099040512
lat_01_mod -39.79939482865696
lat_01_mod -39.722522933497494
lat_01_mod -39.64556514937795
lat_01_mod -39.56852171746229
lat_01_mod -39.49139248397716
lat_01_mod -39.414177690984324
lat_01_mod -39.33687718649035
lat_01_mod -39.25949121346409
lat_01_mod -39.18201962169648
lat_01_mod -39.10446265507271
lat_01_mod -39.026820165172495
lat_01_mod -38.94909239680651
lat_01_mod -38.87127920334764
lat_01_mod -38.79338083054117
lat_01_mod -38.71539713355743
lat_01_mod -38.6373283590854
lat_01_mod -38.55917436409714
lat_01_mod -38.480935396234294
lat_01_mod -38.402611314274864
lat_01_mod -38.32420236682212
lat_01_mod -38.24570841446415
lat_01_mod -38.16712970677471
lat_01_mod -38.0884661061561
lat_01_mod -38.00971786316136
lat_01_mod -37.93088484201106
lat_01_mod -37.851967294246286
lat_01_mod -37.772965085909846
lat_01_mod -37.693878469539534
lat_01_mod -37.61470731300435
lat_01_mod -37.53545186984742
lat_01_mod -37.45611200976781
lat_

lat_01_mod -16.42049687015551
lat_01_mod -16.32455195620721
lat_01_mod -16.228559883221106
lat_01_mod -16.13252105633506
lat_01_mod -16.036435564073468
lat_01_mod -15.940303812710189
lat_01_mod -15.84412589220787
lat_01_mod -15.747902209967636
lat_01_mod -15.651632857377699
lat_01_mod -15.555318242957538
lat_01_mod -15.458958459508088
lat_01_mod -15.36255391665811
lat_01_mod -15.26610470860825
lat_01_mod -15.16961124608732
lat_01_mod -15.073073624682506
lat_01_mod -14.976492256213264
lat_01_mod -14.879867237639997
lat_01_mod -14.783198981863242
lat_01_mod -14.686487587203125
lat_01_mod -14.589733467631538
lat_01_mod -14.492936722814687
lat_01_mod -14.39609776778593
lat_01_mod -14.299216703543747
lat_01_mod -14.202293946172919
lat_01_mod -14.10532959799024
lat_01_mod -14.008324076121701
lat_01_mod -13.911277484188298
lat_01_mod -13.814190240346873
lat_01_mod -13.71706244950835
lat_01_mod -13.6198945308499
lat_01_mod -13.522686590557955
lat_01_mod -13.425439048819337
lat_01_mod -13.32815

lat_01_mod 10.785733218933103
lat_01_mod 10.883950603275439
lat_01_mod 10.982135547454417
lat_01_mod 11.080287929023381
lat_01_mod 11.178407315709944
lat_01_mod 11.27649358612572
lat_01_mod 11.374546308855647
lat_01_mod 11.47256536358603
lat_01_mod 11.570550319771696
lat_01_mod 11.668501058189939
lat_01_mod 11.766417149177911
lat_01_mod 11.864298474620044
lat_01_mod 11.962144605748115
lat_01_mod 12.059955425569713
lat_01_mod 12.157730506223396
lat_01_mod 12.25546973185579
lat_01_mod 12.35317267552425
lat_01_mod 12.450839222530183
lat_01_mod 12.548468946861613
lat_01_mod 12.646061734990322
lat_01_mod 12.743617161846737
lat_01_mod 12.841135115088473
lat_01_mod 12.938615170599938
lat_01_mod 13.036057217239895
lat_01_mod 13.133460831858166
lat_01_mod 13.230825904529832
lat_01_mod 13.328152013081416
lat_01_mod 13.425439048819337
lat_01_mod 13.522686590557955
lat_01_mod 13.6198945308499
lat_01_mod 13.71706244950835
lat_01_mod 13.814190240346873
lat_01_mod 13.911277484188298
lat_01_mod 14.008

In [11]:
temp_z0_1_mod_xr = xr.DataArray(temp_z0_1_mod.transpose('yt_ocean', 'xt_ocean', 'year'), name='temp_z0_1_mod',
                        coords=[lat_1_mod, lon_1_mod, year], 
                        dims=['yt_ocean', 'xt_ocean', 'year'])
print(temp_z0_1_mod_xr)
temp_z0_1_mod_xr.to_netcdf(output_path + 'temp_z0_1_mod.nc')

temp_z0_1_slope_mod_xr = xr.DataArray(temp_z0_1_slope_mod, name='temp_z0_1_slope_mod',
                        coords=[lat_1_mod, lon_1_mod], 
                        dims=['yt_ocean', 'xt_ocean'])
print(temp_z0_1_slope_mod_xr)
temp_z0_1_slope_mod_xr.to_netcdf(output_path + 'temp_z0_1_slope_mod.nc')

temp_z0_1_p_value_mod_xr = xr.DataArray(temp_z0_1_p_value_mod, name='temp_z0_1_p_value_mod',
                        coords=[lat_1_mod, lon_1_mod], 
                        dims=['yt_ocean', 'xt_ocean'])
print(temp_z0_1_p_value_mod_xr)
temp_z0_1_p_value_mod_xr.to_netcdf(output_path + 'temp_z0_1_p_value_mod.nc')

temp_z0_1_std_err_mod_xr = xr.DataArray(temp_z0_1_std_err_mod, name='temp_z0_1_std_err_mod',
                        coords=[lat_1_mod, lon_1_mod], 
                        dims=['yt_ocean', 'xt_ocean'])
print(temp_z0_1_std_err_mod_xr)
temp_z0_1_std_err_mod_xr.to_netcdf(output_path + 'temp_z0_1_std_err_mod.nc')



temp_z0_025_mod_xr = xr.DataArray(temp_z0_025_mod.transpose('yt_ocean', 'xt_ocean', 'year'), name='temp_z0_025_mod',
                        coords=[lat_025_mod, lon_025_mod, year], 
                        dims=['yt_ocean', 'xt_ocean', 'year'])
print(temp_z0_025_mod_xr)
temp_z0_025_mod_xr.to_netcdf(output_path + 'temp_z0_025_mod.nc')

temp_z0_025_slope_mod_xr = xr.DataArray(temp_z0_025_slope_mod, name='temp_z0_025_slope_mod',
                        coords=[lat_025_mod, lon_025_mod], 
                        dims=['yt_ocean', 'xt_ocean'])
print(temp_z0_025_slope_mod_xr)
temp_z0_025_slope_mod_xr.to_netcdf(output_path + 'temp_z0_025_slope_mod.nc')

temp_z0_025_p_value_mod_xr = xr.DataArray(temp_z0_025_p_value_mod, name='temp_z0_025_p_value_mod',
                        coords=[lat_025_mod, lon_025_mod], 
                        dims=['yt_ocean', 'xt_ocean'])
print(temp_z0_025_p_value_mod_xr)
temp_z0_025_p_value_mod_xr.to_netcdf(output_path + 'temp_z0_025_p_value_mod.nc')

temp_z0_025_std_err_mod_xr = xr.DataArray(temp_z0_025_std_err_mod, name='temp_z0_025_std_err_mod',
                        coords=[lat_025_mod, lon_025_mod], 
                        dims=['yt_ocean', 'xt_ocean'])
print(temp_z0_025_std_err_mod_xr)
temp_z0_025_std_err_mod_xr.to_netcdf(output_path + 'temp_z0_025_std_err_mod.nc')



temp_z0_01_mod_xr = xr.DataArray(temp_z0_01_mod.transpose('yt_ocean', 'xt_ocean', 'year'), name='temp_z0_01_mod',
                        coords=[lat_01_mod, lon_01_mod, year], 
                        dims=['yt_ocean', 'xt_ocean', 'year'])
print(temp_z0_01_mod_xr)
temp_z0_01_mod_xr.to_netcdf(output_path + 'temp_z0_01_mod.nc')

temp_z0_01_slope_mod_xr = xr.DataArray(temp_z0_01_slope_mod, name='temp_z0_01_slope_mod',
                        coords=[lat_01_mod, lon_01_mod], 
                        dims=['yt_ocean', 'xt_ocean'])
print(temp_z0_01_slope_mod_xr)
temp_z0_01_slope_mod_xr.to_netcdf(output_path + 'temp_z0_01_slope_mod.nc')

temp_z0_01_p_value_mod_xr = xr.DataArray(temp_z0_01_p_value_mod, name='temp_z0_01_p_value_mod',
                        coords=[lat_01_mod, lon_01_mod], 
                        dims=['yt_ocean', 'xt_ocean'])
print(temp_z0_01_p_value_mod_xr)
temp_z0_01_p_value_mod_xr.to_netcdf(output_path + 'temp_z0_01_p_value_mod.nc')

temp_z0_01_std_err_mod_xr = xr.DataArray(temp_z0_01_std_err_mod, name='temp_z0_01_std_err_mod',
                        coords=[lat_01_mod, lon_01_mod], 
                        dims=['yt_ocean', 'xt_ocean'])
print(temp_z0_01_std_err_mod_xr)
temp_z0_01_std_err_mod_xr.to_netcdf(output_path + 'temp_z0_01_std_err_mod.nc')

<xarray.DataArray 'temp_z0_1_mod' (yt_ocean: 160, xt_ocean: 90, year: 14)>
array([[[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        ...,
        [-1.6798401, -1.3712769, -1.2894592, ..., -1.7720337,
         -1.6766357, -1.5481567],
        [-1.6969604, -1.3629761, -1.2753296, ..., -1.7737122,
         -1.6951599, -1.5387268],
        [-1.7177734, -1.3548584, -1.2640076, ..., -1.7743225,
         -1.7155151, -1.525177 ]],

       [[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        ...,
        [-1.5954285, -1.4140625, 

<xarray.DataArray 'temp_z0_025_slope_mod' (yt_ocean: 475, xt_ocean: 360)>
array([[        nan,         nan,         nan, ..., -0.00813015,
        -0.00802485, -0.00797991],
       [        nan,         nan,         nan, ..., -0.00797367,
        -0.00786884, -0.00789238],
       [        nan,         nan,         nan, ..., -0.00782571,
        -0.00781109, -0.00791163],
       ...,
       [ 0.025143  ,  0.02376172,  0.02259132, ...,  0.00756333,
         0.00858751,  0.00951518],
       [ 0.02401928,  0.02348096,  0.02202531, ...,  0.0073868 ,
         0.00789446,  0.00870288],
       [ 0.02333106,  0.02145292,  0.02114962, ...,  0.0071224 ,
         0.00730068,  0.00758238]])
Coordinates:
  * yt_ocean  (yt_ocean) float64 -69.99 -69.88 -69.78 ... 19.49 19.72 19.96
  * xt_ocean  (xt_ocean) float64 90.12 90.38 90.62 90.88 ... 179.4 179.6 179.9
<xarray.DataArray 'temp_z0_025_p_value_mod' (yt_ocean: 475, xt_ocean: 360)>
array([[       nan,        nan,        nan, ..., 0.37875587, 0.385617